## Documentation

To read more about the search API, visit the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-your-data.html) and [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html).

![search_api_docs](../images/search_api_docs.png)

## Connect to ElasticSearch


In [1]:
import { Client } from "npm:@elastic/elasticsearch";
import { load } from "https://deno.land/std/dotenv/mod.ts";

const env = await load({ envPath: "../.env" });

const client = new Client({
  node: env.ELASTICSEARCH_NODE,
  auth: {
    apiKey: env.ELASTICSEARCH_API_KEY,
  },
});

const info = await client.info();
console.log(info);


{
  name: "1e6d9c093984",
  cluster_name: "docker-cluster",
  cluster_uuid: "v3fUyW9OReext6IjPiOCqg",
  version: {
    number: "8.17.4",
    build_flavor: "default",
    build_type: "docker",
    build_hash: "c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1",
    build_date: "2025-03-20T15:39:59.811110136Z",
    build_snapshot: false,
    lucene_version: "9.12.0",
    minimum_wire_compatibility_version: "7.17.0",
    minimum_index_compatibility_version: "7.0.0"
  },
  tagline: "You Know, for Search"
}


## Create indices


In [ ]:
await client.indices.delete({ index: "my_index", ignore_unavailable: true });
await client.indices.create({ index: "my_index" });

await client.indices.delete({ index: "my_index_2", ignore_unavailable: true });
await client.indices.create({ index: "my_index_2" });


{ acknowledged: true, shards_acknowledged: true, index: "my_index_2" }

Let's index the documents sequentially in both indices.

In [5]:
import data from "../data/dummy_data.json" with { type: "json" };

async function insertDocument(index: string, document: any) {
  return await client.index({
    index,
    document,
  });
}

data.forEach(async (document: any) => {
  await insertDocument("my_index", document);
  await insertDocument("my_index_2", document);
});


## Searching

In [ ]:
const response = await client.search({
  index: "my_index",
  query: {
    match_all: {},
  },
});

console.log(
  `The number of documents in my_index is: `,
  response["hits"]["total"]["value"]
);


The number of documents in my_index is:  3


In [ ]:
const response = await client.search({
  index: "my_index_2",
  query: {
    match_all: {},
  },
});

console.log(
  `The number of documents in my_index_2 is: `,
  response["hits"]["total"]["value"]
);


The number of documents in my_index_2 is:  3


Or we can provide the `index` argument multiple indices at once.

In [ ]:
const response = await client.search({
  index: "my_index,my_index_2",
  query: {
    match_all: {},
  },
});

console.log(
  `The number of documents in my_index and my_index_2 is: `,
  response["hits"]["total"]["value"]
);


The number of documents in my_index and my_index_2 is:  6


We can also use wildcards `*` to match multiple indices without listing them individually, such as `"my_index*"`.

In [ ]:
const response = await client.search({
  index: "my_index*",
  query: {
    match_all: {},
  },
});

console.log(
  `The number of documents in all indices starting with 'my_index' is: `,
  response["hits"]["total"]["value"]
);


The number of documents in all indices starting with 'my_index' is:  6


Or, to search all indices, we use `_all`.

In [16]:
const response = await client.search({
  index: "_all",
  query: {
    match_all: {},
  },
});

console.log(
  `The number of documents in all indices is: `,
  response["hits"]["total"]["value"]
);


The number of documents in all indices is:  6
